# TargetPanelBench Demo
This notebook demonstrates how to load the synthetic dataset, run the baseline methods, and compare their performance to the precomputed results from the proprietary Adaptive Ensemble Algorithm (AEA).

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))
import pandas as pd
import json
from evaluation import compute_ranking_metrics, panel_recall, panel_diversity_score, build_adjacency
from panel_design import select_top_k, select_diverse_panel
from baselines.simple_score_rank import run_simple_baseline
from baselines.cma_es import run_cma_es_baseline

# Load dataset
targets = pd.read_csv('../data/targets.csv')
edges = pd.read_csv('../data/ppi_edges.csv')
adjacency = build_adjacency(edges)


In [ ]:
# Run the simple sum-of-features baseline
simple_results = run_simple_baseline('../data/targets.csv', '../data/ppi_edges.csv', panel_size=12, k=20)
print('Simple baseline metrics:')
print(simple_results['metrics'])
print('Panel recall:', simple_results['panel_recall'])
print('Panel diversity:', simple_results['panel_diversity'])


In [ ]:
# Run the evolutionary baseline
cma_results = run_cma_es_baseline('../data/targets.csv', '../data/ppi_edges.csv', iterations=50, objective='ndcg', panel_size=12, k=20, seed=42)
print('CMA-ES baseline metrics:')
print(cma_results['metrics'])
print('Panel recall:', cma_results['panel_recall'])
print('Panel diversity:', cma_results['panel_diversity'])


In [ ]:
# Load precomputed AEA results
with open('../results/aea_results.json') as f:
    aea_results = json.load(f)
print('AEA metrics:')
print(aea_results['metrics'])
print('Panel recall:', aea_results['panel_recall'])
print('Panel diversity:', aea_results['panel_diversity'])


In [ ]:
# Compare metrics from all methods
import pandas as pd
metrics_df = pd.DataFrame({
    'Method': ['Simple', 'CMA-ES', 'AEA'],
    'Precision@20': [simple_results['metrics']['precision_at_20'], cma_results['metrics']['precision_at_20'], aea_results['metrics']['precision_at_20']],
    'MRR': [simple_results['metrics']['mrr'], cma_results['metrics']['mrr'], aea_results['metrics']['mrr']],
    'nDCG@20': [simple_results['metrics']['ndcg_at_20'], cma_results['metrics']['ndcg_at_20'], aea_results['metrics']['ndcg_at_20']],
    'Panel Recall': [simple_results['panel_recall'], cma_results['panel_recall'], aea_results['panel_recall']],
    'Panel Diversity': [simple_results['panel_diversity'], cma_results['panel_diversity'], aea_results['panel_diversity']]
})
metrics_df